In [1]:
!pwd

/home/dell/af/DM


In [2]:
from unimol_tools import MolTrain, MolPredict
import pandas as pd
import numpy as np
import math
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import spearmanr
import os

2024-09-20 15:32:58 | unimol_tools/weights/weighthub.py | 17 | INFO | Uni-Mol Tools | Weights will be downloaded to default directory: /home/dell/miniconda3/envs/unimol/lib/python3.10/site-packages/unimol_tools-0.1.0.post2-py3.10.egg/unimol_tools/weights


In [3]:
# 更改工作目录到 `af` 目录
os.chdir('/home/dell/af')

# 验证当前工作目录
print(os.getcwd())

/home/dell/af


In [9]:
weights_path = r"./Unimol_weights/DM/0902_bs_32_lr_1e-4"
data_path = r"./Data4pred/gen_mol_0621_energy_level_screening_1400_for_dm_pred.csv"
save_path = r"./Data4pred/gen_mol_0621_energy_level_screening_1400_DM.csv"
input_csv = r"./tmp.csv"

In [10]:
data = pd.read_csv(data_path)
data.head(5)

,SMILES
0,O=C(O)c1ccc2c3c(ccc(P(=O)(O)O)c13)C(=O)N(CC1=C...
1,O=P(O)(O)CC1=C(c2ccccc2N(c2ccccc2)c2ccccc2)C=C1
2,O=P(O)(O)Cc1ccc(N(c2ccccc2)c2ccccc2)cc1F
3,O=P(O)(O)Cc1cc(-c2ccccc2N(c2ccccc2)c2ccccc2)sc1F
4,O=P(O)(O)c1cccc(N(c2ccccc2)c2ccccc2)c1


In [11]:
pd.DataFrame(data.iloc[:,0]).to_csv(input_csv, index=False)

In [12]:
clf = MolPredict(load_model=weights_path)

In [13]:
# 基于SMILES的文件输入模式的预测 
test_pred = clf.predict(data=input_csv)

2024-09-20 15:35:25 | unimol_tools/data/conformer.py | 89 | INFO | Uni-Mol Tools | Start generating conformers...
1419it [03:42,  6.37it/s]
2024-09-20 15:39:08 | unimol_tools/data/conformer.py | 93 | INFO | Uni-Mol Tools | Failed to generate conformers for 0.00% of molecules.
2024-09-20 15:39:08 | unimol_tools/data/conformer.py | 95 | INFO | Uni-Mol Tools | Failed to generate 3d conformers for 2.04% of molecules.
2024-09-20 15:39:10 | unimol_tools/models/unimol.py | 120 | INFO | Uni-Mol Tools | Loading pretrained weights from /home/dell/miniconda3/envs/unimol/lib/python3.10/site-packages/unimol_tools-0.1.0.post2-py3.10.egg/unimol_tools/weights/mol_pre_no_h_220816.pt
2024-09-20 15:39:11 | unimol_tools/models/nnmodel.py | 206 | INFO | Uni-Mol Tools | start predict NNModel:unimolv1
2024-09-20 15:39:12 | unimol_tools/tasks/trainer.py | 300 | INFO | Uni-Mol Tools | load model success!
2024-09-20 15:39:17 | unimol_tools/tasks/trainer.py | 300 | INFO | Uni-Mol Tools | load model success!
2024

In [14]:
os.remove(input_csv)

In [15]:
test_results = pd.DataFrame({
    "smiles": clf.datahub.data["smiles"],
    "pred": test_pred.flatten(),
    })
test_results.head(10)

,smiles,pred
0,O=C(O)c1ccc2c3c(ccc(P(=O)(O)O)c13)C(=O)N(CC1=C...,3.586676
1,O=P(O)(O)CC1=C(c2ccccc2N(c2ccccc2)c2ccccc2)C=C1,3.756025
2,O=P(O)(O)Cc1ccc(N(c2ccccc2)c2ccccc2)cc1F,4.569075
3,O=P(O)(O)Cc1cc(-c2ccccc2N(c2ccccc2)c2ccccc2)sc1F,4.293637
4,O=P(O)(O)c1cccc(N(c2ccccc2)c2ccccc2)c1,4.957133
5,O=P(O)(O)c1ccccc1N(c1ccccc1)c1ccccc1,5.015993
6,O=[PH](O)(O)=Cc1ccc(N(c2ccccc2)c2ccccc2)cc1,4.762547
7,O=C(O)C(=Cc1ccc(N(c2ccccc2)c2ccccc2)cc1)P(=O)(O)O,7.528917
8,O=P(O)(O)C/C(=C\c1ccc(N(c2ccccc2)c2ccccc2)cc1)...,7.155074
9,O=P(O)(O)/C(=C\c1ccc(N(c2ccccc2)c2ccccc2)cc1)C...,7.825933


In [16]:
test_results.to_csv(save_path, index=False)